# Тестирование различных провайдеров LLM

Поддерживаемые провайдеры:
- **Qwen** - локальный/удаленный Qwen API
- **YandexGPT** - Yandex Cloud LLM API
- **ChatGPT** - OpenAI API
- **Gemini** - Google Gemini API


In [1]:
import sys
import time
from pathlib import Path
from typing import Any

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

print(f"📁 Корневая директория проекта: {project_root}")

📁 Корневая директория проекта: /Users/avolon/Documents/SIRIUS/T-bank_NLP_


In [2]:
from tplexity.llm_client import get_llm

print("✅ Импорты выполнены успешно")

✅ Импорты выполнены успешно


In [3]:
TEST_MESSAGES = [
    {
        "role": "system",
        "content": "Ты - полезный ассистент.",
    },
    {
        "role": "user",
        "content": "Привет!",
    },
]

GENERATION_PARAMS = {"temperature": 0.7, "max_tokens": 500}

print("Тестовый запрос:")
print(f"Системный промпт: {TEST_MESSAGES[0]['content']}")
print(f"Сообщение пользователя: {TEST_MESSAGES[1]['content']}")
print(f"Параметры генерации: {GENERATION_PARAMS}")

Тестовый запрос:
Системный промпт: Ты - полезный ассистент.
Сообщение пользователя: Привет!
Параметры генерации: {'temperature': 0.7, 'max_tokens': 500}


In [4]:
async def test_provider(
    provider: str,
    messages: list[dict],
    generation_params: dict,
) -> dict[str, Any]:
    """
    Тестирует работу LLM провайдера на тестовом запросе

    Args:
        provider (str): Название провайдера (qwen, yandexgpt, chatgpt, gemini)
        messages (list[dict]): Список сообщений в формате OpenAI (системный промпт + сообщение пользователя)
        generation_params (dict): Параметры генерации

    Returns:
        Словарь с результатами тестирования
    """
    result = {"provider": provider, "status": "unknown", "response": None, "time": 0, "error": None}

    start_time = time.time()

    try:
        print(f"Тестирование провайдера: {provider.upper()}")

        try:
            client = get_llm(provider)
            print(f"✅ Клиент инициализирован: model={client.model}")
        except Exception as e:
            error_msg = f"Ошибка инициализации клиента: {e}"
            result["error"] = error_msg
            result["status"] = "failed"
            print(f"❌ {error_msg}")
            return result

        try:
            response = await client.generate(
                messages=messages,
                temperature=generation_params.get("temperature"),
                max_tokens=generation_params.get("max_tokens"),
            )

            result["response"] = response
            result["status"] = "success"
            result["time"] = time.time() - start_time

            print(f"✅ Успешно за {result['time']:.2f}с")
            print(f"Ответ: {response}")

        except Exception as e:
            result["status"] = "failed"
            result["error"] = str(e)
            result["time"] = time.time() - start_time
            print(f"❌ Ошибка: {e}")

    except Exception as e:
        result["status"] = "failed"
        result["error"] = f"Критическая ошибка: {e}"
        result["time"] = time.time() - start_time
        print(f"❌ Критическая ошибка: {e}")

    return result

In [5]:
PROVIDERS = ["qwen", "yandexgpt", "chatgpt", "gemini"]

all_results = {}

for provider in PROVIDERS:
    result = await test_provider(
        provider=provider,
        messages=TEST_MESSAGES,
        generation_params=GENERATION_PARAMS,
    )
    all_results[provider] = result
    print()

print("✅ Тестирование завершено")

Тестирование провайдера: QWEN
✅ Клиент инициализирован: model=QuantTrio/Qwen3-VL-30B-A3B-Instruct-AWQ
✅ Успешно за 1.03с
Ответ: Привет! Как я могу тебе помочь? 😊

Тестирование провайдера: YANDEXGPT
✅ Клиент инициализирован: model=gpt://b1g737e6b62eqjfkjd8l/yandexgpt-lite
✅ Успешно за 1.53с
Ответ: Здравствуйте! Чем я могу вам помочь?

Тестирование провайдера: CHATGPT
✅ Клиент инициализирован: model=gpt-4o-mini
✅ Успешно за 3.53с
Ответ: Привет! Как я могу помочь тебе сегодня?

Тестирование провайдера: GEMINI
✅ Клиент инициализирован: model=gemini-2.5-flash


❌ [llm_client] Ошибка при вызове LLM: Error code: 400 - [{'error': {'code': 400, 'message': 'User location is not supported for the API use.', 'status': 'FAILED_PRECONDITION'}}]


❌ Ошибка: Error code: 400 - [{'error': {'code': 400, 'message': 'User location is not supported for the API use.', 'status': 'FAILED_PRECONDITION'}}]

✅ Тестирование завершено
